In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from functools import partial
from random import random
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict
from collections import Counter
import math

In [ ]:
df = load_iris()
df.data.shap

In [ ]:

def f_hash(w,r,b,x):
    return int((np.dot(w,x)+b)/r)
def euclideanDistance(instance1, instance2):
    distance = 0
    for x in range(instance1.shape[0]):
        distance += pow(instance1[x] - instance2[0][x], 2)
    return math.sqrt(distance)

In [ ]:
class KNNHash(object):
    def __init__(self,m,L,nn):
        self.m = m
        self.L = L
        self.nn = nn

    def fit(self,X,y):
        self.t_hh = [] #hash table
        for j in range(self.L):
            f_hh = [] #compositional hash function
            for i in range(self.m):
                w = np.random.rand(1,X[0].shape[0]) #  weights of a hash function
                f_hh.append(partial(f_hash,w = w,r=random(),b=random())) # list of initialized hash function
            self.t_hh.append(
                (defaultdict(list),f_hh)
            )
        for n in range(X.shape[0]): 
            for j in range(self.L):
                ind = 0
                for i in range(self.m):
                    ind = ind + self.t_hh[j][1][i](x=X[n]) #calculation of index in hash table, simply sum of all hash func
                self.t_hh[j][0][ind].append((X[n],y[n])) #saving sample into corresponding index
    
    def predict(self,u):
        
        for j in range(self.L):
            inds = []
            distances = []
            level_distance = []
            for i in range(self.m):
                inds.append(self.t_hh[j][1][i](x=u))
            cntr = Counter([outp for inpt,outp in self.t_hh[j][0][sum(inds)]])
            print(cntr)
            for i in self.t_hh[j][0][sum(inds)]:
                distances.append(euclideanDistance(u,i))
                level_distance.append(i[1])
        ans = distances.index(min(distances))
        print(min(distances))
        print("answer:" + str(level_distance[ans]))
     

In [ ]:
scaler = MinMaxScaler()
scaler.fit(df.data)
x = scaler.transform(df.data)
y = df.target

In [ ]:
knnhash = KNNHash(4,4,4)
test1x = x[0]
test2x = x[75]
test3x = x[149]

test1y = y[0]
test2y = y[75]
test3y = y[149]
x = np.delete(x,[0,75,149],axis=0)
y = np.delete(y,[0,75,149],axis=0)

knnhash.fit(x,y)
print(test1y)
knnhash.predict(test1x)
print("-------------")
print(test2y)
knnhash.predict(test2x)
print("-------------")
print(test3y)
knnhash.predict(test3x)